# Retrieval

In [1]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
paths = [
    "./data/Aprendizado_de_máquina.pdf",
    "./data/Inteligência_artificial_generativa.pdf",
    "./data/Inteligência_artificial.pdf"
]

pages = []

for path in paths:
    loader = PyPDFLoader(path)
    pages.extend(loader.load())

doc_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = doc_split.split_documents(pages)

In [3]:
len(documents)

268

## Modificando o metadata

In [4]:
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('./data/', '')
    doc.metadata['doc_id'] = i

In [5]:
documents[0].metadata

{'producer': 'Skia/PDF m128',
 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/128.0.0.0 Safari/537.36',
 'creationdate': '2025-02-18T18:05:54+00:00',
 'title': 'Aprendizado de máquina – Wikipédia, a enciclopédia livre',
 'moddate': '2025-02-18T18:05:54+00:00',
 'source': 'Aprendizado_de_máquina.pdf',
 'total_pages': 14,
 'page': 0,
 'page_label': '1',
 'doc_id': 0}

## Criação do VectorStore

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

embed_model = OpenAIEmbeddings()

In [7]:
directory = "./data/chroma_retrieval"

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embed_model,
    persist_directory=directory
)

## Semantic Search

In [8]:
pergunta = "O que é Inteligência artificial?"

docs = vectordb.similarity_search(pergunta, k=3)

for doc in docs:
    print(doc.page_content)
    print(f'\nMETADATA: {doc.metadata}\n')
    print(f'===============================\n')

Inteligência artificial
Na informática, a Inteligência Artificial (IA) genericamente é a inteligência, o raciocínio e, o
aprendizado exibida por máquinas semelhante ao raciocino humano;[1][2] busca desenvolver máquinas
autônomas ou sistemas especialistas capazes de simular o pensamento humano e realizar varias tarefas
complexas de forma independente.[3][4][5][6][7] É o sistema que permite aos computadores executar
funções avançadas, como a capacidade de analisar dados em grande escala e fazer

METADATA: {'creationdate': '2025-02-18T18:05:19+00:00', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/128.0.0.0 Safari/537.36', 'doc_id': 139, 'moddate': '2025-02-18T18:05:19+00:00', 'page': 0, 'page_label': '1', 'producer': 'Skia/PDF m128', 'source': 'Inteligência_artificial.pdf', 'title': 'Inteligência artificial – Wikipédia, a enciclopédia livre', 'total_pages': 18}


Inteligência artificial
Na informática, a Inteligência Artificial (IA) gene

## MMR

In [9]:
pergunta = "O que é Inteligência artificial?"

docs = vectordb.max_marginal_relevance_search(pergunta, k=3, fetch_k=10)

for doc in docs:
    print(doc.page_content)
    print(f'\nMETADATA: {doc.metadata}\n')
    print(f'===============================\n')

Inteligência artificial
Na informática, a Inteligência Artificial (IA) genericamente é a inteligência, o raciocínio e, o
aprendizado exibida por máquinas semelhante ao raciocino humano;[1][2] busca desenvolver máquinas
autônomas ou sistemas especialistas capazes de simular o pensamento humano e realizar varias tarefas
complexas de forma independente.[3][4][5][6][7] É o sistema que permite aos computadores executar
funções avançadas, como a capacidade de analisar dados em grande escala e fazer

METADATA: {'creationdate': '2025-02-18T18:05:19+00:00', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/128.0.0.0 Safari/537.36', 'doc_id': 139, 'moddate': '2025-02-18T18:05:19+00:00', 'page': 0, 'page_label': '1', 'producer': 'Skia/PDF m128', 'source': 'Inteligência_artificial.pdf', 'title': 'Inteligência artificial – Wikipédia, a enciclopédia livre', 'total_pages': 18}


Obtida de "https://pt.wikipedia.org/w/index.php?title=Inteligência_artifici

## Filtragem

In [10]:
pergunta = "O que é Inteligência artificial?"

docs = vectordb.similarity_search(pergunta, k=3, filter={'source': 'Inteligência_artificial_generativa.pdf'})

for doc in docs:
    print(doc.page_content)
    print(f'\nMETADATA: {doc.metadata}\n')
    print(f'===============================\n')

Inteligência artificial generativa
Inteligência artificial generativa ou IA generativa é um ramo da inteligência artificial focado na
criação de conteúdo novo, baseado em padrões identificados nos conjuntos de dados de treinamento. É
possível criar conteúdo como texto, imagens, áudio, vídeo, código de software, podendo ser gerados a
partir de solicitações em linguagem natural, ou até de através de imagens para imagens, e de vídeos para

METADATA: {'creationdate': '2025-02-18T18:06:24+00:00', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/128.0.0.0 Safari/537.36', 'doc_id': 94, 'moddate': '2025-02-18T18:06:24+00:00', 'page': 0, 'page_label': '1', 'producer': 'Skia/PDF m128', 'source': 'Inteligência_artificial_generativa.pdf', 'title': 'Inteligência artificial generativa – Wikipédia, a enciclopédia livre', 'total_pages': 6}


Inteligência artificial generativa
Inteligência artificial generativa ou IA generativa é um ramo da inteligência 

## LLM Aided Retrieval

In [11]:
from langchain_openai.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo

In [12]:
metadata_info = [
    AttributeInfo(
        name='source',
        description='Nome da apostila de onde o texto original foi retirado. Deve ter o valor de: Inteligência_artificial_generativa.pdf ou Inteligência_artificial.pdf',
        type='string'
    ),
    AttributeInfo(
        name='page',
        description='A página da apostila de onde o texto se origina',
        type='integer'
    )
]

In [13]:
document_description = 'Arquivos do Wikipedia'

llm = OpenAI()

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_description,
    metadata_info,
    verbose=True
)

In [15]:
pergunta = "Qual a relação entre IA e GenAI?"

docs = retriever.get_relevant_documents(pergunta)

for doc in docs:
    print(doc.page_content)
    print(f'\nMETADATA: {doc.metadata}\n')
    print(f'===============================\n')

n/OnlinePubs-Nils/General%20Essays/AIMag04-04-002.pdf) (PDF). AI Magazine. 1 (1).
ISSN 0738-4602 (https://www.worldcat.org/issn/0738-4602)
34. Nilsson, Nils (1998). Artificial Intelligence: A New Synthesis. San Francisco: Morgan
Kaufmann Publishers. p. 7. ISBN 978-1-55860-467-4
35. Pei Wang (2008). Artificial general intelligence, 2008: proceedings of the First AGI
Conference (http://books.google.com/books?id=a_ZR81Z25z0C&pg=PA63). [S.l.]: IOS

METADATA: {'creationdate': '2025-02-18T18:05:19+00:00', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/128.0.0.0 Safari/537.36', 'doc_id': 252, 'moddate': '2025-02-18T18:05:19+00:00', 'page': 15, 'page_label': '16', 'producer': 'Skia/PDF m128', 'source': 'Inteligência_artificial.pdf', 'title': 'Inteligência artificial – Wikipédia, a enciclopédia livre', 'total_pages': 18}


n/OnlinePubs-Nils/General%20Essays/AIMag04-04-002.pdf) (PDF). AI Magazine. 1 (1).
ISSN 0738-4602 (https://www.worldcat.org/